<a href="https://colab.research.google.com/github/rssubramaniyan1/EVA8/blob/main/EVA8_Assignment%203_Part%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Architechture

3 Convolution layers with batch norm and the 3rd layer has dropout of 0.1
1 Max pool layer

1 Transition layer of 1D Convolution

2 Convolution layers with batch norm followed by a maxpool

1 Transition layer of 1D Convolution

Output layer of 1 convolution and a 1FC layer

Total of 6594 parameters with highest accuracy at 99.22

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary


torch.manual_seed(1)
batch_size = 64
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [2]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # input - 28x28x1, output - 26x26x16, RF - 3x3
        self.conv1 = nn.Sequential(nn.Conv2d(1, 8, 3, padding=0), nn.ReLU(), nn.BatchNorm2d(8))
        # input - 26x26x16, output - 24x24x16, RF - 5x5
        self.conv2 = nn.Sequential(nn.Conv2d(8,8, 3, padding=0), nn.ReLU(), nn.BatchNorm2d(8))
        # input - 24x24x16, output - 22x22x16, RF - 7x7 dropout - 0.1
        self.conv3 = nn.Sequential(nn.Conv2d(8, 16, 3, padding=0), nn.ReLU(), nn.BatchNorm2d(16), nn.Dropout(0.1))

        # Max Pooling
        self.pool1 = nn.MaxPool2d(2, 2)  # output 11x11x16

        # Transition layer
        self.conv4 = nn.Sequential(nn.Conv2d(16, 8, 1, padding=0))  # output 11x11x8

        # Convolution block 2
        self.conv5 = nn.Sequential(nn.Conv2d(8, 8, 3, padding=0), nn.ReLU(), nn.BatchNorm2d(8)) # output 9x9x8
        self.conv6 = nn.Sequential(nn.Conv2d(8, 16, 3, padding=0), nn.ReLU(), nn.BatchNorm2d(16))  # input -9x9 OUtput- 7x7
        self.pool2 = nn.MaxPool2d(2, 2)  # output 3x3x16

        # Transition layer
        self.conv7 = nn.Sequential(nn.Conv2d(16, 16, 1, padding=0))  # output 11x11x8

        self.conv8 = nn.Sequential(nn.Conv2d(16, 16, 3, padding=0))  # input -7x7 OUtput- 5x5
        #self.conv9 = nn.Sequential(nn.Conv2d(8, 16, 3, padding=0))  # input -5x5 OUtput- 3x3
        # GAP
        #self.gap = nn.Sequential(nn.AvgPool2d(kernel_size=3))  # output 1x1x16
        self.fc1 = nn.Linear(16*1*1, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.pool1(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.pool2(x)
        x = self.conv7(x)
        x = self.conv8(x)
        #x = self.conv9(x)
        #x = self.gap(x)
        x = x.view(-1, 16*1*1)
        x = self.fc1(x)
        return F.log_softmax(x)

In [3]:
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

<ipython-input-2-ff994d8177a7>:46: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
            Conv2d-4            [-1, 8, 24, 24]             584
              ReLU-5            [-1, 8, 24, 24]               0
       BatchNorm2d-6            [-1, 8, 24, 24]              16
            Conv2d-7           [-1, 16, 22, 22]           1,168
              ReLU-8           [-1, 16, 22, 22]               0
       BatchNorm2d-9           [-1, 16, 22, 22]              32
          Dropout-10           [-1, 16, 22, 22]               0
        MaxPool2d-11           [-1, 16, 11, 11]               0
           Conv2d-12            [-1, 8, 11, 11]             136
           Conv2d-13              [-1, 8, 9, 9]             584
             ReLU-14              [-1, 

loss=0.17220737040042877 batch_id=937: 100%|██████████| 938/938 [01:01<00:00, 15.34it/s]



Test set: Average loss: 0.0795, Accuracy: 9743/10000 (97.4300%)



loss=0.15610241889953613 batch_id=937: 100%|██████████| 938/938 [01:01<00:00, 15.27it/s]


...


Test set: Average loss: 0.0326, Accuracy: 9907/10000 (99.0700%)



loss=8.266120858024806e-05 batch_id=937: 100%|██████████| 938/938 [01:01<00:00, 15.21it/s]



Test set: Average loss: 0.0337, Accuracy: 9900/10000 (99.0000%)



loss=0.06217896193265915 batch_id=937: 100%|██████████| 938/938 [01:00<00:00, 15.41it/s]



Test set: Average loss: 0.0296, Accuracy: 9922/10000 (99.2200%)

